# EDA of Patient Data
# 02_EDA_Patient

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 28/09/2025 | Adrienne | Created   | Moved Martin's code to this program | 
| |  |  | |

# Content

* [Introduction](#introduction)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json_lines
import seaborn as sns

In [ ]:
path = "../data/clean"
claim = pd.read_pickle(f"{path}/claim_clean.pkl")
# claim_old = pd.read_pickle(f"{path}/claim_old.pkl")
# claim_resp = pd.read_pickle(f"{path}/claim_response.pkl")
patient = pd.read_pickle(f"{path}/patient.pkl")
eob = pd.read_pickle(f"{path}/explanation_of_benefit.pkl")
# coverage = pd.read_pickle(f"{path}/coverage.pkl")

In [ ]:
# Select relevant columns for initial exploration
patient = patient[['address_0_state', 'birthDate', 'deceasedBoolean', 'extension_1_valueCoding_display', 'extension_3_valueDate', 'gender', 'id', 'identifier_1_value', 'deceasedBoolean', 'deceasedDateTime', 'address_0_postalCode']]
patient = patient.rename({
  'extension_1_valueCoding_display': 'race',
  'extension_3_valueDate': 'referenceYear',
  'identifier_1_value': 'patient_medicare_number',
  'address_0_postal_code': 'postalCode'
}, axis=1)

### Exploring a Patient

In [ ]:
# patient_id = -10000000000322
medicare_number = "1S00E00AA50"

In [ ]:
c = claim[claim['patient_medicare_number'] == medicare_number]
# e = eob[eob['patient'] == str(patient_id)]
# p = patient[patient['identifier_1_value'] == medicare_number]

In [ ]:
# Checking if there are any non-na DRGs with HCPCS codes
c[(~c['drg'].isna()) & (~c['hcpcs_0'].isna())]

In [ ]:
c[c['claim_id'] == '-100007150']